In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from logicpy import *
from logicpy.core import *
from logicpy.builtin import *
from logicpy.predicate import *
from logicpy.data import *
a = _

In [2]:
u, n = Universe().and_namespace()

n.bla[a.X] = a.X << 5
#n.bla[a.bar] = True
#n.bla[a.both(a.X, a.Y)] = n.bla(a.X) & n.bla(a.Y)

#[res for res, _ in zip(u.query(n.bla(a.Z)), range(3))]
u.simple_query(n.bla(a.Z), debug=True)

-> PredicateCall bla(Z:996) with ok
   -> Clause bla/1 with ok
substitute Z:996 -> X:316
replaced_E = set(), True
      -> Eval (X:316 << 5) with {Z:996 = X:316}
         -> Eval (X:316 << 5) with {X:316 = 5, Z:996 = X:316}
            -> DoMgu DoMgu with {X:316 = 5, Z:996 = X:316}
substitute Z:996 -> X:316
replaced_E = {(Variable('X', 6942119964251049147), 5)}, True
substitute X:316 -> 5
replaced_E = {(Variable('Z', -1), Variable('X', 6942119964251049147))}, True
            <- DoMgu DoMgu with {X:316 = 5, Z:996 = X:316}
substitute Z:996 -> X:316
replaced_E = {(Variable('X', 6942119964251049147), 5)}, True
substitute X:316 -> 5
replaced_E = {(Variable('Z', -1), Variable('X', 6942119964251049147))}, True


[{'Z': Variable('X', 6942119964251049147)}]

In [12]:
Z = Variable('Z', 5)
X = Variable('X', 7)
replace(X, X, 5)

Variable('X', 7)